In [1]:
# IMPORTAR DATASET LIMPIO DESDE EXCEL
import pandas as pd
import numpy as np
from datetime import datetime

# Importar la hoja "Datos_Limpios"
df = pd.read_excel('Accidentes_Transito_Limpio.xlsx', sheet_name='Datos_Limpios')

print("=== IMPORTACIÓN DESDE EXCEL ===")
print(f"Registros importados: {len(df)}")
print(f"Columnas: {len(df.columns)}")
print("\nNombres de columnas:")
print(df.columns.tolist())

print("\n=== VERIFICAR DATOS DE FECHAS ===")
print("Primeras 5 fechas y horas:")
print(df[['fecha_accidente', 'hora_accidente']].head())

print("\nTipos de datos actuales:")
print(df[['fecha_accidente', 'hora_accidente']].dtypes)

print("\nValores únicos de fecha (primeros 10):")
print(sorted(df['fecha_accidente'].unique())[:10])

print("\nValores únicos de hora (primeros 10):")
print(df['hora_accidente'].dropna().unique()[:10])

print("\n=== VERIFICAR VALORES FALTANTES ===")
print(f"Fechas faltantes: {df['fecha_accidente'].isnull().sum()}")
print(f"Horas faltantes: {df['hora_accidente'].isnull().sum()}")

print("\n✅ Dataset importado correctamente")
print("🕐 Listo para procesar fechas y horas")

=== IMPORTACIÓN DESDE EXCEL ===
Registros importados: 8155
Columnas: 9

Nombres de columnas:
['fecha_corte', 'fecha_accidente', 'hora_accidente', 'departamento_ubicacion', 'codigo_carretera', 'kilometro_via', 'tipo_accidente', 'num_fallecidos', 'num_heridos']

=== VERIFICAR DATOS DE FECHAS ===
Primeras 5 fechas y horas:
   fecha_accidente hora_accidente
0         20200101          05:40
1         20200101          16:30
2         20200101          07:45
3         20200101          18:30
4         20200101          18:40

Tipos de datos actuales:
fecha_accidente     int64
hora_accidente     object
dtype: object

Valores únicos de fecha (primeros 10):
[20200101, 20200102, 20200103, 20200104, 20200105, 20200106, 20200107, 20200108, 20200109, 20200110]

Valores únicos de hora (primeros 10):
['05:40' '16:30' '07:45' '18:30' '18:40' '16:35' '21:45' '23:50' '02:30'
 '23:40']

=== VERIFICAR VALORES FALTANTES ===
Fechas faltantes: 0
Horas faltantes: 88

✅ Dataset importado correctamente
🕐 Listo

In [2]:
# TRANSFORMAR FECHAS Y CREAR VARIABLES TEMPORALES

print("=== TRANSFORMACIÓN DE FECHAS ===")

# 1. CONVERTIR fecha_accidente de formato numérico (20200101) a datetime
df['fecha_accidente_dt'] = pd.to_datetime(df['fecha_accidente'], format='%Y%m%d')

print("✅ Fecha convertida a datetime")
print("Ejemplo de conversión:")
print(df[['fecha_accidente', 'fecha_accidente_dt']].head(3))

# 2. CREAR VARIABLES DERIVADAS DE FECHA
df['año'] = df['fecha_accidente_dt'].dt.year
df['mes'] = df['fecha_accidente_dt'].dt.month
df['dia'] = df['fecha_accidente_dt'].dt.day
df['dia_semana'] = df['fecha_accidente_dt'].dt.dayofweek  # 0=Lunes, 6=Domingo
df['nombre_dia'] = df['fecha_accidente_dt'].dt.day_name()
df['nombre_mes'] = df['fecha_accidente_dt'].dt.month_name()
df['trimestre'] = df['fecha_accidente_dt'].dt.quarter

print("\n=== VARIABLES TEMPORALES CREADAS ===")
print("✅ año, mes, dia, dia_semana, nombre_dia, nombre_mes, trimestre")

# 3. PROCESAR HORA
print("\n=== PROCESAMIENTO DE HORAS ===")

# Convertir hora a datetime (solo para extraer componentes)
df['hora_accidente_dt'] = pd.to_datetime(df['hora_accidente'], format='%H:%M', errors='coerce').dt.time

# Extraer componentes de hora
df['hora'] = pd.to_datetime(df['hora_accidente'], format='%H:%M', errors='coerce').dt.hour
df['minuto'] = pd.to_datetime(df['hora_accidente'], format='%H:%M', errors='coerce').dt.minute

# Crear categorías de hora
def categorizar_hora(hora):
    if pd.isna(hora):
        return 'No especificado'
    elif 0 <= hora < 6:
        return 'Madrugada (0-6h)'
    elif 6 <= hora < 12:
        return 'Mañana (6-12h)'
    elif 12 <= hora < 18:
        return 'Tarde (12-18h)'
    else:
        return 'Noche (18-24h)'

df['periodo_dia'] = df['hora'].apply(categorizar_hora)

print("✅ Variables de hora creadas: hora, minuto, periodo_dia")

# 4. VERIFICAR RESULTADOS
print("\n=== VERIFICACIÓN DE TRANSFORMACIONES ===")
print("Columnas temporales nuevas:")
columnas_temporales = ['fecha_accidente_dt', 'año', 'mes', 'dia', 'dia_semana', 
                      'nombre_dia', 'nombre_mes', 'trimestre', 'hora', 'periodo_dia']
print(columnas_temporales)

print("\nPrimeras 5 filas con nuevas variables:")
print(df[['fecha_accidente_dt', 'año', 'mes', 'nombre_dia', 'hora', 'periodo_dia']].head())

# 5. ESTADÍSTICAS TEMPORALES BÁSICAS
print("\n=== ANÁLISIS TEMPORAL BÁSICO ===")
print("Rango de fechas:")
print(f"Desde: {df['fecha_accidente_dt'].min()}")
print(f"Hasta: {df['fecha_accidente_dt'].max()}")

print("\nDistribución por año:")
print(df['año'].value_counts().sort_index())

print("\nDistribución por mes:")
print(df['mes'].value_counts().sort_index())

print("\nDistribución por día de la semana:")
print(df['nombre_dia'].value_counts())

print("\nDistribución por período del día:")
print(df['periodo_dia'].value_counts())

print("\n✅ Transformaciones temporales completadas")

=== TRANSFORMACIÓN DE FECHAS ===
✅ Fecha convertida a datetime
Ejemplo de conversión:
   fecha_accidente fecha_accidente_dt
0         20200101         2020-01-01
1         20200101         2020-01-01
2         20200101         2020-01-01

=== VARIABLES TEMPORALES CREADAS ===
✅ año, mes, dia, dia_semana, nombre_dia, nombre_mes, trimestre

=== PROCESAMIENTO DE HORAS ===
✅ Variables de hora creadas: hora, minuto, periodo_dia

=== VERIFICACIÓN DE TRANSFORMACIONES ===
Columnas temporales nuevas:
['fecha_accidente_dt', 'año', 'mes', 'dia', 'dia_semana', 'nombre_dia', 'nombre_mes', 'trimestre', 'hora', 'periodo_dia']

Primeras 5 filas con nuevas variables:
  fecha_accidente_dt   año  mes nombre_dia  hora       periodo_dia
0         2020-01-01  2020    1  Wednesday   5.0  Madrugada (0-6h)
1         2020-01-01  2020    1  Wednesday  16.0    Tarde (12-18h)
2         2020-01-01  2020    1  Wednesday   7.0    Mañana (6-12h)
3         2020-01-01  2020    1  Wednesday  18.0    Noche (18-24h)
4      

In [3]:
# CORREGIR IDIOMA - CONVERTIR DÍAS Y MESES AL ESPAÑOL

print("=== ANTES DE LA CORRECCIÓN ===")
print("Días en inglés:")
print(df['nombre_dia'].value_counts())
print("\nMeses en inglés:")
print(df['nombre_mes'].value_counts())

# Diccionarios de traducción
dias_español = {
    'Monday': 'Lunes',
    'Tuesday': 'Martes', 
    'Wednesday': 'Miércoles',
    'Thursday': 'Jueves',
    'Friday': 'Viernes',
    'Saturday': 'Sábado',
    'Sunday': 'Domingo'
}

meses_español = {
    'January': 'Enero',
    'February': 'Febrero',
    'March': 'Marzo',
    'April': 'Abril',
    'May': 'Mayo',
    'June': 'Junio',
    'July': 'Julio',
    'August': 'Agosto',
    'September': 'Septiembre',
    'October': 'Octubre',
    'November': 'Noviembre',
    'December': 'Diciembre'
}

# Aplicar las traducciones
df['nombre_dia'] = df['nombre_dia'].map(dias_español)
df['nombre_mes'] = df['nombre_mes'].map(meses_español)

print("\n=== DESPUÉS DE LA CORRECCIÓN ===")
print("Días en español:")
print(df['nombre_dia'].value_counts())
print("\nMeses en español:")
print(df['nombre_mes'].value_counts())

# Verificar que la corrección funcionó
print("\n=== VERIFICACIÓN ===")
print("Primeras 5 filas con nombres corregidos:")
print(df[['fecha_accidente_dt', 'año', 'mes', 'nombre_dia', 'nombre_mes', 'hora', 'periodo_dia']].head())

print("\n✅ Nombres corregidos al español exitosamente")
print("🇪🇸 Ahora tienes días y meses en español")

=== ANTES DE LA CORRECCIÓN ===
Días en inglés:
nombre_dia
Saturday     1255
Monday       1250
Sunday       1204
Wednesday    1128
Friday       1109
Thursday     1106
Tuesday      1103
Name: count, dtype: int64

Meses en inglés:
nombre_mes
January      961
February     905
August       861
September    828
July         787
March        732
June         684
May          602
April        494
December     484
November     430
October      387
Name: count, dtype: int64

=== DESPUÉS DE LA CORRECCIÓN ===
Días en español:
nombre_dia
Sábado       1255
Lunes        1250
Domingo      1204
Miércoles    1128
Viernes      1109
Jueves       1106
Martes       1103
Name: count, dtype: int64

Meses en español:
nombre_mes
Enero         961
Febrero       905
Agosto        861
Septiembre    828
Julio         787
Marzo         732
Junio         684
Mayo          602
Abril         494
Diciembre     484
Noviembre     430
Octubre       387
Name: count, dtype: int64

=== VERIFICACIÓN ===
Primeras 5 filas con no

In [4]:
# EXPORTAR VERSIÓN FINAL CON FECHAS PROCESADAS

# Crear el diccionario de datos ACTUALIZADO con todas las variables
diccionario_final = {
    'Variable': [
        'fecha_corte',
        'fecha_accidente', 
        'hora_accidente',
        'departamento_ubicacion',
        'codigo_carretera',
        'kilometro_via',
        'tipo_accidente',
        'num_fallecidos',
        'num_heridos',
        'fecha_accidente_dt',
        'año',
        'mes',
        'dia',
        'dia_semana',
        'nombre_dia',
        'nombre_mes',
        'trimestre',
        'hora',
        'minuto',
        'periodo_dia'
    ],
    'Descripción': [
        'Fecha de corte de la información (formato numérico)',
        'Fecha del accidente de tránsito (formato numérico)',
        'Hora del accidente de tránsito (formato texto HH:MM)',
        'Departamento donde se localizó el accidente de tránsito',
        'Código de la carretera en la que se reportó el accidente',
        'Kilómetro de la vía en la que se reportó el accidente',
        'Modalidad del accidente: atropello, choque, despiste, especial, volcadura',
        'Número de personas reportadas como fallecidas',
        'Número de personas reportadas como heridas',
        'Fecha del accidente convertida a formato datetime',
        'Año del accidente (2020-2021)',
        'Mes del accidente (1-12)',
        'Día del mes del accidente (1-31)',
        'Día de la semana numérico (0=Lunes, 6=Domingo)',
        'Nombre del día de la semana en español',
        'Nombre del mes en español',
        'Trimestre del año (1-4)',
        'Hora del accidente (0-23)',
        'Minuto del accidente (0-59)',
        'Período del día: Madrugada, Mañana, Tarde, Noche'
    ],
    'Tipo_de_Dato': [
        'Entero',
        'Entero',
        'Texto',
        'Texto',
        'Texto',
        'Decimal',
        'Texto',
        'Decimal',
        'Decimal',
        'Datetime',
        'Entero',
        'Entero',
        'Entero',
        'Entero',
        'Texto',
        'Texto',
        'Entero',
        'Decimal',
        'Decimal',
        'Texto'
    ],
    'Valores_Faltantes': [
        0,
        0,
        88,
        7,
        46,
        45,
        28,
        3,
        10,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        88,
        88,
        0
    ]
}

# Convertir a DataFrame
dict_final_df = pd.DataFrame(diccionario_final)

# Estadísticas actualizadas del dataset
estadisticas_finales = {
    'Estadística': [
        'Total de registros',
        'Total de variables',
        'Período de datos',
        'Rango de fechas',
        'Ceros en num_fallecidos',
        'Porcentaje ceros num_fallecidos',
        'Ceros en num_heridos', 
        'Porcentaje ceros num_heridos',
        'Media num_fallecidos',
        'Media num_heridos',
        'Máximo num_fallecidos',
        'Máximo num_heridos',
        'Años incluidos',
        'Meses incluidos',
        'Días de semana incluidos',
        'Variables temporales creadas'
    ],
    'Valor': [
        len(df),
        len(df.columns),
        '2020-2021',
        f"{df['fecha_accidente_dt'].min().strftime('%d/%m/%Y')} - {df['fecha_accidente_dt'].max().strftime('%d/%m/%Y')}",
        (df['num_fallecidos'] == 0).sum(),
        f"{((df['num_fallecidos'] == 0).sum()/len(df))*100:.2f}%",
        (df['num_heridos'] == 0).sum(),
        f"{((df['num_heridos'] == 0).sum()/len(df))*100:.2f}%",
        f"{df['num_fallecidos'].mean():.3f}",
        f"{df['num_heridos'].mean():.3f}",
        df['num_fallecidos'].max(),
        df['num_heridos'].max(),
        ', '.join(map(str, sorted(df['año'].unique()))),
        'Enero a Diciembre',
        'Lunes a Domingo',
        '10 variables nuevas'
    ]
}

estadisticas_df = pd.DataFrame(estadisticas_finales)

# Análisis temporal adicional
analisis_temporal = {
    'Dimensión_Temporal': [
        'Distribución por Año',
        '2020',
        '2021',
        'Día más frecuente',
        'Mes más frecuente',
        'Período del día más frecuente',
        'Trimestre más frecuente'
    ],
    'Detalle': [
        'Cantidad de accidentes por año',
        df[df['año'] == 2020].shape[0],
        df[df['año'] == 2021].shape[0],
        df['nombre_dia'].mode()[0],
        df['nombre_mes'].mode()[0],
        df['periodo_dia'].mode()[0],
        df['trimestre'].mode()[0]
    ]
}

temporal_df = pd.DataFrame(analisis_temporal)

# Exportar a Excel con múltiples hojas
nombre_archivo_final = 'Accidentes_Transito_FINAL_ConFechas.xlsx'

with pd.ExcelWriter(nombre_archivo_final, engine='openpyxl') as writer:
    # Hoja 1: Dataset completo con fechas
    df.to_excel(writer, sheet_name='Datos_Completos', index=False)
    
    # Hoja 2: Diccionario actualizado
    dict_final_df.to_excel(writer, sheet_name='Diccionario_Completo', index=False)
    
    # Hoja 3: Estadísticas generales
    estadisticas_df.to_excel(writer, sheet_name='Estadisticas_Generales', index=False)
    
    # Hoja 4: Análisis temporal
    temporal_df.to_excel(writer, sheet_name='Analisis_Temporal', index=False)

print("✅ EXPORTACIÓN FINAL COMPLETADA")
print(f"📁 Archivo creado: {nombre_archivo_final}")
print("\n📊 CONTENIDO DEL ARCHIVO:")
print("📋 Hoja 1: 'Datos_Completos' - Dataset con fechas procesadas (8,155 registros, 20 variables)")
print("📋 Hoja 2: 'Diccionario_Completo' - Descripción de TODAS las variables (20 variables)")
print("📋 Hoja 3: 'Estadisticas_Generales' - Resumen estadístico completo")
print("📋 Hoja 4: 'Analisis_Temporal' - Patrones temporales identificados")

print("\n🎯 VARIABLES PRINCIPALES PARA TU TESIS:")
print("⭐ num_fallecidos: Variable principal para exceso de ceros")
print("⭐ num_heridos: Variable principal para exceso de ceros")
print("📅 Variables temporales: Para análisis de patrones estacionales")

print(f"\n📈 RESUMEN DEL EXCESO DE CEROS:")
print(f"• num_fallecidos: {((df['num_fallecidos'] == 0).sum()/len(df))*100:.1f}% de ceros")
print(f"• num_heridos: {((df['num_heridos'] == 0).sum()/len(df))*100:.1f}% de ceros")

print("\n✅ Dataset FINAL listo para análisis completo de exceso de ceros")

✅ EXPORTACIÓN FINAL COMPLETADA
📁 Archivo creado: Accidentes_Transito_FINAL_ConFechas.xlsx

📊 CONTENIDO DEL ARCHIVO:
📋 Hoja 1: 'Datos_Completos' - Dataset con fechas procesadas (8,155 registros, 20 variables)
📋 Hoja 2: 'Diccionario_Completo' - Descripción de TODAS las variables (20 variables)
📋 Hoja 3: 'Estadisticas_Generales' - Resumen estadístico completo
📋 Hoja 4: 'Analisis_Temporal' - Patrones temporales identificados

🎯 VARIABLES PRINCIPALES PARA TU TESIS:
⭐ num_fallecidos: Variable principal para exceso de ceros
⭐ num_heridos: Variable principal para exceso de ceros
📅 Variables temporales: Para análisis de patrones estacionales

📈 RESUMEN DEL EXCESO DE CEROS:
• num_fallecidos: 88.3% de ceros
• num_heridos: 42.3% de ceros

✅ Dataset FINAL listo para análisis completo de exceso de ceros
